# Coding Attention Mechanisms

### Attending to different parts of the input with self-attention

In [1]:
import torch

In [2]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your     (x^1)
        [0.55, 0.87, 0.66], # journey  (x^2)
        [0.57, 0.85, 0.64], # starts   (x^3)
        [0.22, 0.58, 0.33], # with     (x^4)
        [0.77, 0.25, 0.10], # one      (x^5)
        [0.05, 0.80, 0.55]
   ]
)

In [3]:
inputs.shape

torch.Size([6, 3])

In [4]:
# We compute attention scores by applying dot product against each other element for a query
query = inputs[1]   # "journey"
attn_scores = torch.empty(inputs.shape[0])  # Faster than torch.zeros
for idx, query_i in enumerate(inputs):
    attn_scores[idx] = torch.dot(query, query_i)

print("Attention Scores:", attn_scores)
print("Sum:", attn_scores.sum())

Attention Scores: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
Sum: tensor(6.5617)


> **Nota importante:** Si nos fijamos, la operación de producto escalar es la misma a nivel efectivo para todas las posiciones de los input embeds. **Es por ello que, si queremos portar algo de información posicional se necesita usar una entrada con embeddings posicionales (relativos o absolutos)**.

In [5]:
# We then normalize the scores for trainig stability (remember vanishing or exploding activations in deep nets)
attn_weights_2_tmp = attn_scores / attn_scores.sum()
print("Attention Weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention Weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [6]:
# Better to use softmax for extreme values. Specifically better to use pytorch imp
# attn_weights = torch.softmax(attn_scores, dim=0) # Sames as below
attn_weights_2 = attn_scores.softmax(dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [7]:
inputs.shape[1] # Columns

3

In [8]:
# Context vector will be the weighed sum of input embeddings based on attention scores:
context_vector = torch.zeros(inputs.shape[1])
for idx, input in enumerate(inputs):
    context_vector += input * attn_weights_2[idx]

context_vector

tensor([0.4419, 0.6515, 0.5683])

> We have to compute this context vector for each input embedding. Since we are computing first the attention scores, we can calculate this as matrix multiplications

In [9]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [10]:
# Then for normalization
attn_weights = attn_scores.softmax(dim=1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [11]:
# Finally, to get the context vectors
context_vectors = attn_weights @ inputs
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

### Self attention with trainable weights